<a href="https://colab.research.google.com/github/aloumon/CS5542/blob/master/Spark_ICP_2_finalized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [6]:
!pip install pyspark


     |████████████████████████████████| 204.2MB 66kB/s 
     |████████████████████████████████| 204kB 32.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=ade0da7b3f6459690d36d6be8dd213fa27dfedb35039a28e254a45abd49d5b9b
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


Configuring a SparkSession

The entry point to using Spark SQL is an object called SparkSession. It initiates a Spark Application which all the code for that Session will run on.

.builder — gives access to Builder API which is used to configure the session .

.master() — determines where the program will run; "local[*]" sets it to run locally on all cores but you can use "local[1]" to run on one core for example. In this case, our programs will be run on Google’s servers.

.appName() — optional method to name the Spark Application

.getOrCreate() — gets an existing SparkSession or creates new one if none exists

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Big_Data_Application_ICP_2").getOrCreate()

To open a local file on Google Colab you need to run the following code which will prompt you to select a file from your computer:

In [8]:
from google.colab import files
files.upload()

Saving I_cp2.csv to I_cp2.csv


{'I_cp2.csv': b'The,The,In,Virus,New,Silvio,A,University,C.D.C.,"Iowa,",fallout,studies,"Berlusconi,",judge,of,tells,college,from,show,Italy\x92s,orders,South,health,students,the,inexpensive,former,the,officials,staged,Sturgis,steroid,prime,University,Carolina,reports,to,a,motorcycle,drugs,"minister,",of,that,be,"sickout,",rally:,can,tests,California,more,ready,and,A,help,positive.,to,than,to,a,death,critically,,stop\r\n'}

Now load our data into a Spark DataFrame using the .read.csv()

In [9]:
data = spark.read.csv('I_cp2.csv',inferSchema=True, header=True)

Data Exploration

Now let’s move into understanding how we can get more familiar with our data!
The first thing we can do is check the shape of of our DataFrame. Unlike Pandas, there is no dedicated method for this but we can use the .count() and .columns() to retrieve the information ourselves.
The .count() method returns the number of rows in the DataFrame and .columns returns a list of column names.

In [10]:
data.count(), len(data.columns)

(0, 63)

Viewing DataFrames
To view a DataFrame, use the .show() method:

In [11]:
data.show()

+----+----+---+-----+---+------+---+-----------+------+-----+-------+-------+-----------+-----+----+-----+-------+----+----+-------+------+-----+------+--------+-----+-----------+------+-----+---------+------+-------+-------+-----+------------+--------+-------+----+---+----------+-----+---------+----+----+---+--------+------+---+-----+----------+----+-----+---+---+----+---------+----+----+----+---+-----+----------+----+----+
|The0|The1| In|Virus|New|Silvio| A6|University7|C.D.C.|Iowa,|fallout|studies|Berlusconi,|judge|of14|tells|college|from|show|Italy�s|orders|South|health|students|the24|inexpensive|former|the27|officials|staged|Sturgis|steroid|prime|University33|Carolina|reports|to36|a37|motorcycle|drugs|minister,|of41|that| be|sickout,|rally:|can|tests|California|more|ready|and|A52|help|positive.|to55|than|to57|a58|death|critically|_c61|stop|
+----+----+---+-----+---+------+---+-----------+------+-----+-------+-------+-----------+-----+----+-----+-------+----+----+-------+------+---

As you can see, running data.show(5) displayed the first 5 rows of our DataFrame, along with the header. Calling .show() with no parameters will return the first 20 records.


Let’s see what our data is comprised of using the .printSchema() method (alternatively you can use .dtypes):

In [12]:
data.dtypes

[('The0', 'string'),
 ('The1', 'string'),
 ('In', 'string'),
 ('Virus', 'string'),
 ('New', 'string'),
 ('Silvio', 'string'),
 ('A6', 'string'),
 ('University7', 'string'),
 ('C.D.C.', 'string'),
 ('Iowa,', 'string'),
 ('fallout', 'string'),
 ('studies', 'string'),
 ('Berlusconi,', 'string'),
 ('judge', 'string'),
 ('of14', 'string'),
 ('tells', 'string'),
 ('college', 'string'),
 ('from', 'string'),
 ('show', 'string'),
 ('Italy�s', 'string'),
 ('orders', 'string'),
 ('South', 'string'),
 ('health', 'string'),
 ('students', 'string'),
 ('the24', 'string'),
 ('inexpensive', 'string'),
 ('former', 'string'),
 ('the27', 'string'),
 ('officials', 'string'),
 ('staged', 'string'),
 ('Sturgis', 'string'),
 ('steroid', 'string'),
 ('prime', 'string'),
 ('University33', 'string'),
 ('Carolina', 'string'),
 ('reports', 'string'),
 ('to36', 'string'),
 ('a37', 'string'),
 ('motorcycle', 'string'),
 ('drugs', 'string'),
 ('minister,', 'string'),
 ('of41', 'string'),
 ('that', 'string'),
 ('be', 

We can also selectively choose which columns we want to display with the .select() method.
Included the truncate=False parameter that adjusts the size of columns to prevent values from being cut off

In [21]:
data.collect()

[]

In [25]:
from itertools import groupby 

d_list = ['The0','The1','In','Virus','New','Silvio','A6','University','C.D.C.','Iowa','fallout','studies','Berlusconi','judge','of','tells','college','from','show','Italy','s','orders','South','health','students','the','inexpensive','former','the','officials','staged','Sturgis','steroid','prime','University','Carolina','reports','to','motorcycle','drugs','minister','of','that','be','sickout','rally','can','tests','California','more','ready','and','A','help','positive','to','than','to','a','death','critically','stop']
util_func = lambda x: x[0] 
temp = sorted(d_list, key = util_func) 
res = [list(ele) for i, ele in groupby(temp, util_func)] 
print("The list after Categorization : " + str(res)) 


The list after Categorization : [['A6', 'A'], ['Berlusconi'], ['C.D.C.', 'Carolina', 'California'], ['In', 'Iowa', 'Italy'], ['New'], ['Silvio', 'South', 'Sturgis'], ['The0', 'The1'], ['University', 'University'], ['Virus'], ['and', 'a'], ['be'], ['college', 'can', 'critically'], ['drugs', 'death'], ['fallout', 'from', 'former'], ['health', 'help'], ['inexpensive'], ['judge'], ['motorcycle', 'minister', 'more'], ['of', 'orders', 'officials', 'of'], ['prime', 'positive'], ['reports', 'rally', 'ready'], ['studies', 'show', 's', 'students', 'staged', 'steroid', 'sickout', 'stop'], ['tells', 'the', 'the', 'to', 'that', 'tests', 'to', 'than', 'to']]
